In [1]:
import requests
import pandas as pd
from pathlib import Path
import os
import io

In [2]:
base_url = "https://www.football-data.co.uk"
en_base_url = f"{base_url}/mmz4281"
en_2425_url = f"{en_base_url}/2425v"
en_2425_pl_url = f"{en_2425_url}/E0.csv"

In [50]:
resp = requests.get(en_2425_pl_url)
data = resp.content.decode('utf-8')

df = pd.read_csv(io.StringIO(data))
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,16/08/2024,20:00,Man United,Fulham,1,0,H,0,0,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,17/08/2024,12:30,Ipswich,Liverpool,0,2,A,0,0,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,17/08/2024,15:00,Arsenal,Wolves,2,0,H,1,0,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,17/08/2024,15:00,Everton,Brighton,0,3,A,0,1,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,17/08/2024,15:00,Newcastle,Southampton,1,0,H,1,0,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10


In [9]:
type(df.columns.values)
df.columns.values

array(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
       'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST',
       'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D',
       'B365A', 'BWH', 'BWD', 'BWA', 'BFH', 'BFD', 'BFA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', '1XBH', '1XBD', '1XBA', 'MaxH', 'MaxD',
       'MaxA', 'AvgH', 'AvgD', 'AvgA', 'BFEH', 'BFED', 'BFEA', 'B365>2.5',
       'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5',
       'Avg<2.5', 'BFE>2.5', 'BFE<2.5', 'AHh', 'B365AHH', 'B365AHA',
       'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'BFEAHH',
       'BFEAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA',
       'BFCH', 'BFCD', 'BFCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD',
       'WHCA', '1XBCH', '1XBCD', '1XBCA', 'MaxCH', 'MaxCD', 'MaxCA',
       'AvgCH', 'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA', 'B365C>2.5',
       'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5',


In [10]:
df["HomeTeam"].unique()

array(['Man United', 'Ipswich', 'Arsenal', 'Everton', 'Newcastle',
       "Nott'm Forest", 'West Ham', 'Brentford', 'Chelsea', 'Leicester',
       'Brighton', 'Crystal Palace', 'Fulham', 'Man City', 'Southampton',
       'Tottenham', 'Aston Villa', 'Bournemouth', 'Wolves', 'Liverpool'],
      dtype=object)

## Extract data for one team

In [12]:
# dft['Date'] = pd.to_datetime(dft['Date'], format='%d/%m/%Y')
# dft = dft.sort_values(by = 'Date', ascending=True)

In [14]:
# from datetime import datetime

# dft.drop('Rest', inplace=True, axis=1)

# dft['Rest'] = datetime.now() - dft.Date
# dft


# teams = df.HomeTeam.unique().tolist()
# teams
# df.index = pd.MultiIndex.from_arrays([df.HomeTeam, df.Date], )
# df = df.sort_index()
# df.loc["Arsenal"]

In [76]:
def num_of_wins(x):
    return (x == 'H').sum()
    
def num_of_not_wins(x):
    return (x != 'H').sum()
    
    
dfth = df.groupby(["HomeTeam", "Div"]).agg(
    HT_P=('Div', 'count'), 
    HT_FTWH=('FTR', num_of_wins), 
    HT_FTLDH=('FTR', num_of_not_wins), 
    # HT_FTWDA=?
    # HT_FTLA=?
    HT_HTWH=('HTR', num_of_wins), 
    HT_HTLDH = ('HTR', num_of_not_wins), 
    # HT_HTWDA=?
    # HT_HTLA=?
    # HT_P=?
    # HT_LYD=?
    # HT_LYP=?
    HT_FTHG=('FTHG', 'median'),
    HT_HTHG=('HTHG', 'median'),
    HT_HS=('HS', 'median'),
    HT_HST=('HST', 'median'),
    HT_HC=('HC', 'median'),
    HT_HF=('HF', 'median'),
    HT_HY=('HY', 'median'),
    HT_HR=('HR', 'median'),
    # HT_FTAG=?,
    # HT_HTAG=?,
    
    
)
dfth.head() 

,,HT_P,HT_FTWH,HT_FTLDH,HT_HTWH,HT_HTLDH,HT_FTHG,HT_HTHG,HT_HS,HT_HST,HT_HC,HT_HF,HT_HY,HT_HR
HomeTeam,Div,,,,,,,,,,,,,
Arsenal,E0,2,1,1,2,0,1.5,1.0,14.5,6.5,5.5,14.5,2.5,0.5
Aston Villa,E0,3,2,1,0,3,3.0,0.0,11.0,4.0,6.0,10.0,1.0,0.0
Bournemouth,E0,2,0,2,1,1,0.5,0.5,13.0,4.0,5.5,14.5,2.5,0.0
Brentford,E0,2,2,0,2,0,2.5,1.0,14.5,6.0,3.0,8.0,1.5,0.0
Brighton,E0,3,1,2,2,1,2.0,1.0,14.0,5.0,9.0,12.0,3.0,0.0


## What data is needed

### For the home team:

* HT_P - num of games played in the season home / away
* HT_FTWH - num of wins home full time
* HT_FTLDH - num of loss or draw home full time
* HT_FTWDA - num of wins draw away full time
* HT_FTLA - num of loss away full time
* HT_HTWH - num of wins home half time
* HT_HTLDH - num of loss or draw home half time
* HT_HTWDA - num of wins draw away half time
* HT_HTLA - num of loss away half time
* HT_P = position on the table
* HT_LYD last year division
* HT_LYP last year position on the table
* median - HT_FTHG
* median - HT_HTHG
* median - HT_HS
* median - HT_HST
<!-- * median - HT_HHW -->
* median - HT_HC
* median - HT_HF
<!-- * median - HT_HFKC -->
<!-- * median - HT_HO -->
* median - HT_HY
* median - HT_HR
* median - HT_FTAG
* median - HT_HTAG
* median - HT_AS
* median - HT_AST
<!-- * median - HT_AHW -->
* median - HT_AC
* median - HT_AF
<!-- * median - HT_AFKC -->
<!-- * median - HT_AO -->
* median - HT_AY
* median - HT_AR


## For the away team
* num of games played in the season home / away
* num of wins home / away
* num of wins home / away half time
* position on the table
* last year division
* last year position on the table
* median - AT_FTHG
* median - AT_HTHG
* median - AT_HS
* median - AT_HST
* median - AT_HHW
* median - AT_HC
* median - AT_HF
* median - AT_HFKC
* median - AT_HO
* median - AT_HY
* median - AT_HR
* median - AT_FTAG
* median - AT_HTAG
* median - AT_AS
* median - AT_AST
* median - AT_AHW
* median - AT_AC
* median - AT_AF
* median - AT_AFKC
* median - AT_AO
* median - AT_AY
* median - AT_AR


In [37]:
def num_of_wins(x):
    return (x == 'H').sum()
    
def num_of_not_wins(x):
    return (x != 'H').sum()
    


def count_of_3(x):
    return (x == 3).sum()
    
dfth = df.groupby(["HomeTeam", "Div"]).agg(
    HT_P=('Div', 'count'), 
    HT_FTHG=('FTHG', 'median'),
    HT_
)
dfth 


# result = df.groupby('Category').agg(
#     count=('Value', 'count'),        # Count rows in each group
#     sum_value=('Value', 'sum'),      # Sum of 'Value' column
#     median_other_value=('OtherValue', 'median')  # Median of 'OtherValue' column
# )

,,HT_P,HT_FTHG
HomeTeam,Div,,
Arsenal,E0,2,1.5
Aston Villa,E0,3,3.0
Bournemouth,E0,2,0.5
Brentford,E0,2,2.5
Brighton,E0,3,2.0
Chelsea,E0,2,0.5
Crystal Palace,E0,3,0.0
Everton,E0,2,1.0
Fulham,E0,3,2.0


In [46]:
import pandas as pd

# Example DataFrame
data = {'Category': ['A', 'A', 'B', 'B', 'C'],
        'Value': [10, 20, 30, 40, 50],
        'OtherValue': [1, 2, 3, 3, 5]}
df = pd.DataFrame(data)

# Define the custom function
def count_of_3(x):
    return (x == 3).sum()

# Group by 'Category' and apply multiple aggregation functions including the custom function
result = df.groupby('Category').agg(
    count=('Value', 'count'),               # Count rows in each group
    sum_value=('Value', 'sum'),             # Sum of 'Value' column
    median_other_value=('OtherValue', 'median'),  # Median of 'OtherValue' column
    count_of_3_in_other_value=('OtherValue', count_of_3)  # Custom function: count where OtherValue == 3
)

# Print the result
print(result)


          count  sum_value  median_other_value  count_of_3_in_other_value
Category                                                                 
A             2         30                 1.5                          0
B             2         70                 3.0                          2
C             1         50                 5.0                          0


In [41]:
df

Div        Date   Time        HomeTeam  \
HomeTeam       Date                                                
Arsenal        17/08/2024  E0  17/08/2024  15:00         Arsenal   
               31/08/2024  E0  31/08/2024  12:30         Arsenal   
Aston Villa    14/09/2024  E0  14/09/2024  17:30     Aston Villa   
               24/08/2024  E0  24/08/2024  17:30     Aston Villa   
Bournemouth    14/09/2024  E0  14/09/2024  20:00     Bournemouth   
               25/08/2024  E0  25/08/2024  14:00     Bournemouth   
Brentford      18/08/2024  E0  18/08/2024  14:00       Brentford   
               31/08/2024  E0  31/08/2024  15:00       Brentford   
Brighton       14/09/2024  E0  14/09/2024  15:00        Brighton   
               24/08/2024  E0  24/08/2024  12:30        Brighton   
Chelsea        01/09/2024  E0  01/09/2024  13:30         Chelsea   
               18/08/2024  E0  18/08/2024  16:30         Chelsea   
Crystal Palace 14/09/2024  E0  14/09/2024  15:00  Crystal Palace   
               24/08/2024  E0  24/08/2024  15:00  Crystal Palace   
Everton        17/08/2024  E0  17/08/2024  15:00         Everton   
               31/08/2024  E0  31/08/2024  15:00         Everton   
Fulham         14/09/2024  E0  14/09/2024  15:00          Fulham   
               24/08/2024  E0  24/08/2024  15:00          Fulham   
Ipswich        17/08/2024  E0  17/08/2024  12:30         Ipswich   
               31/08/2024  E0  31/08/2024  15:00         Ipswich   
Leicester      19/08/2024  E0  19/08/2024  20:00       Leicester   
               31/08/2024  E0  31/08/2024  15:00       Leicester   
Liverpool      14/09/2024  E0  14/09/2024  15:00       Liverpool   
               25/08/2024  E0  25/08/2024  16:30       Liverpool   
Man City       14/09/2024  E0  14/09/2024  15:00        Man City   
               24/08/2024  E0  24/08/2024  15:00        Man City   
Man United     01/09/2024  E0  01/09/2024  16:00      Man United   
               16/08/2024  E0  16/08/2024  20:00      Man United   
Newcastle      01/09/2024  E0  01/09/2024  13:30       Newcastle   
               17/08/2024  E0  17/08/2024  15:00       Newcastle   
Nott'm Forest  17/08/2024  E0  17/08/2024  15:00   Nott'm Forest   
               31/08/2024  E0  31/08/2024  15:00   Nott'm Forest   
Southampton    14/09/2024  E0  14/09/2024  12:30     Southampton   
               24/08/2024  E0  24/08/2024  15:00     Southampton   
Tottenham      15/09/2024  E0  15/09/2024  14:00       Tottenham   
               24/08/2024  E0  24/08/2024  15:00       Tottenham   
West Ham       17/08/2024  E0  17/08/2024  17:30        West Ham   
               31/08/2024  E0  31/08/2024  17:30        West Ham   
Wolves         15/09/2024  E0  15/09/2024  16:30          Wolves   
               25/08/2024  E0  25/08/2024  14:00          Wolves   

                                 AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  ...  \
HomeTeam       Date                                                    ...   
Arsenal        17/08/2024          Wolves     2     0   H     1     0  ...   
               31/08/2024        Brighton     1     1   D     1     0  ...   
Aston Villa    14/09/2024         Everton     3     2   H     1     2  ...   
               24/08/2024         Arsenal     0     2   A     0     0  ...   
Bournemouth    14/09/2024         Chelsea     0     1   A     0     0  ...   
               25/08/2024       Newcastle     1     1   D     1     0  ...   
Brentford      18/08/2024  Crystal Palace     2     1   H     1     0  ...   
               31/08/2024     Southampton     3     1   H     1     0  ...   
Brighton       14/09/2024         Ipswich     0     0   D     0     0  ...   
               24/08/2024      Man United     2     1   H     1     0  ...   
Chelsea        01/09/2024  Crystal Palace     1     1   D     1     0  ...   
               18/08/2024        Man City     0     2   A     0     1  ...   
Crystal Palace 14/09/2024       Leicester     2     2   D     0     1  ...   